# Machine Learning | Project 1 | 2021/22

- João Santos, 76912
- João Carvalho, 106310

[Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download)

[Useful code](https://www.kaggle.com/code/kaanboke/beginner-friendly-end-to-end-ml-project-enjoy)

## Visualization

## Imports and Data Load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import plotly 
import plotly.express as px

from sklearn.metrics import mutual_info_score

from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
data=pd.read_csv('healthcare-dataset-stroke-data.csv')
data.drop('id',axis=1,inplace=True) # removes the ID column

print(data.shape)
data.head(5)

(5110, 11)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Data Analysis

### Numerical

In [3]:
# Define some classes
categorical = ['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']

numerical = ['age','avg_glucose_level', 'bmi']

In [4]:
# Check stroke ratios
stroke = data['stroke'].astype(str)
had_stroke = stroke.value_counts(normalize=True)


print(f'No Stoke: {had_stroke[0]*100:.2f}%')
print(f'Stoke: {had_stroke[1]*100:.2f}%')

No Stoke: 95.13%
Stoke: 4.87%


These ratios shows us that we have an unbalanced dataset.

In [6]:
# Check for missing data
def missing (df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['MissingAmount', 'MissingRatio'])
    return missing_values

missing(data)

# TODO deal with missing data

,MissingAmount,MissingRatio
bmi,201,0.039335
gender,0,0.000000
age,0,0.000000
hypertension,0,0.000000
heart_disease,0,0.000000
ever_married,0,0.000000
work_type,0,0.000000
Residence_type,0,0.000000
avg_glucose_level,0,0.000000
smoking_status,0,0.000000


In [7]:
# Get description of numerical data
data[numerical].describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000


In [8]:
# Get skewness
data[numerical].skew()

age                 -0.137059
avg_glucose_level    1.572284
bmi                  1.055340
dtype: float64

Metrics of importance

In [9]:
def cat_mut_inf(series):
    return mutual_info_score(series, data['stroke']) 

df_cat = data[categorical].apply(cat_mut_inf) 
df_cat = df_cat.sort_values(ascending=False).to_frame(name='mutual_info_score') 
df_cat

,mutual_info_score
ever_married,0.006950
work_type,0.006825
hypertension,0.005976
heart_disease,0.005897
smoking_status,0.002750
Residence_type,0.000120
gender,0.000051


In [10]:
data[numerical].corr()

,age,avg_glucose_level,bmi
age,1.000000,0.238171,0.333398
avg_glucose_level,0.238171,1.000000,0.175502
bmi,0.333398,0.175502,1.000000


In [11]:
data.groupby('stroke')[numerical].mean()

,age,avg_glucose_level,bmi
stroke,,,
0,41.971545,104.795513,28.823064
1,67.728193,132.544739,30.471292


In [12]:
data[['age','avg_glucose_level','bmi','stroke']].corr()

,age,avg_glucose_level,bmi,stroke
age,1.000000,0.238171,0.333398,0.245257
avg_glucose_level,0.238171,1.000000,0.175502,0.131945
bmi,0.333398,0.175502,1.000000,0.042374
stroke,0.245257,0.131945,0.042374,1.000000


### Graphical

In [15]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.H4("Histogram"),
        html.P("Category:"),
        dcc.Dropdown(
            id='hist',
            options = categorical+numerical,
            value='gender'
        ),
        dcc.Graph(id="hist-graph"),
    ]),
    html.Div([
        html.H4("Dual Histogram"),
        html.P("Category:"),
        dcc.Dropdown(
            id='dual-hist',
            options = categorical+numerical,
            value='gender'
        ),
        dcc.Graph(id="dual-hist-graph"),
    ]),
    html.Div([
        html.H4("Dual Scatter Plot"),
        html.P("Categories:"),
        dcc.Dropdown(
            id="scatter-1",
            options = numerical,
            value='age'
        ),
        dcc.Dropdown(
            id="scatter-2",
            options = numerical,
            value='bmi'
        ),
        dcc.Graph(id="dual-scatter-graph"),
    ])
])

@app.callback(
    Output("hist-graph", "figure"),
    Input("hist", "value"))
def generate_charts_data(mode):
    fig = px.histogram(data, x=mode, title='Stroke', color_discrete_sequence=["red", "#B6E880"])
    return fig

@app.callback(
    Output("dual-hist-graph", "figure"),
    Input("dual-hist", "value"))
def generate_charts_data(mode):
    fig = px.histogram(data, x=mode, title=mode, color="stroke",color_discrete_sequence=["red", "#B6E880"])
    return fig

@app.callback(
    Output("dual-scatter-graph", "figure"),
    Input("scatter-1", "value"),
    Input("scatter-2", "value"))
def generate_charts_data(cat1,cat2):
    fig = px.scatter(data, x=cat1, y=cat2, title=f'{cat1} and {cat2}', color='stroke', color_discrete_sequence=["red", "#B6E880"],hover_data = data[['stroke']])
    return fig

app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
